<a href="https://colab.research.google.com/github/abkerper/project_voting/blob/main/2020_CLEANED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/DS3001/project_voting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

Cloning into 'project_voting'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 70 (delta 5), reused 3 (delta 3), pack-reused 59
Receiving objects: 100% (70/70), 31.37 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (41/41), done.


In [2]:
data_df = pd.read_csv("./project_voting/data/voting_VA.csv")
data_df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


# Taking Difference Between Republican Democrat

In [3]:
data_rep = data_df[data_df['party'] == 'REPUBLICAN'].copy()  # Ensure we're working on a copy to avoid modifying the original DataFrame

# Filter data for DEMOCRAT party
data_dem = data_df[data_df['party'] == 'DEMOCRAT']

# Ensure the indices of both DataFrames are aligned
data_rep = data_rep.set_index('state')
data_dem = data_dem.set_index('state')

# Calculate the difference in candidate votes
data_rep['difference'] = data_rep['candidatevotes'] - data_dem['candidatevotes']

# Reset the index to keep 'state' as a column
data_rep.reset_index(inplace=True)
data_rep.head()

,state,Unnamed: 0,year,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,difference
0,VIRGINIA,11162,2000,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL,1260
1,VIRGINIA,11166,2000,VA,ALBEMARLE,51003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,18291,36846,20220315,TOTAL,2036
2,VIRGINIA,11170,2000,VA,ALLEGHANY,51005,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,2808,5123,20220315,TOTAL,594
3,VIRGINIA,11174,2000,VA,AMELIA,51007,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,2947,4788,20220315,TOTAL,1193
4,VIRGINIA,11178,2000,VA,AMHERST,51009,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6660,11712,20220315,TOTAL,1848


In [13]:
data_2000r = data_rep[data_rep['year'] == 2000]
data_2004r = data_rep[data_rep['year'] == 2004]
data_2008r = data_rep[data_rep['year'] == 2008]
data_2012r = data_rep[data_rep['year'] == 2012]
data_2016r = data_rep[data_rep['year'] == 2016]
data_2020r = data_rep[data_rep['year'] == 2020]
data_2020r.head()

,state,Unnamed: 0,year,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,difference
670,VIRGINIA,70388,2020,VA,ACCOMACK,51001,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3084,16962,20220315,ABSENTEE,-2411
671,VIRGINIA,70389,2020,VA,ACCOMACK,51001,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,6078,16962,20220315,ELECTION DAY,4006
672,VIRGINIA,70390,2020,VA,ACCOMACK,51001,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,10,16962,20220315,PROVISIONAL,-1
673,VIRGINIA,70400,2020,VA,ALBEMARLE,51003,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,9124,64657,20220315,ABSENTEE,-24292
674,VIRGINIA,70401,2020,VA,ALBEMARLE,51003,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,11587,64657,20220315,ELECTION DAY,2871


In [5]:
data_2000 = data_df[(data_df['year'] == 2000)]
data_2004 = data_df[(data_df['year'] == 2004)]
data_2008 = data_df[(data_df['year'] == 2008)]
data_2012 = data_df[(data_df['year'] == 2012)]
data_2016 = data_df[(data_df['year'] == 2016)]
data_2020 = data_df[(data_df['year'] == 2020)]

# Adding Winners

In [6]:
data_by_year = [data_2000, data_2004, data_2008, data_2012, data_2016, data_2020]
df_wins = pd.DataFrame()

for year in data_by_year:
    wins = pd.DataFrame()
    county_names = year['county_name'].values
    county_names = list(set(county_names))
    for county in county_names:
        county_rows = year[year['county_name'] == county]
        max_votes = county_rows['candidatevotes'].idxmax()
        winning_row = year.loc[max_votes]
        wins = pd.concat([wins, pd.DataFrame([winning_row])], ignore_index=True)
    df_wins = pd.concat([df_wins, wins], ignore_index=True)

data_2000 = df_wins[df_wins['year'] == 2000]
data_2004 = df_wins[df_wins['year'] == 2004]
data_2008 = df_wins[df_wins['year'] == 2008]
data_2012 = df_wins[df_wins['year'] == 2012]
data_2016 = df_wins[df_wins['year'] == 2016]
data_2020 = df_wins[df_wins['year'] == 2020]

In [8]:
data_2020 = pd.merge(data_2020, data_2020r[['county_name', 'difference']], on='county_name', how='inner')
data_2020.head(5)

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,difference
0,70509,2020,VIRGINIA,VA,BLAND,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,3479,20220315,ELECTION DAY,663
1,70509,2020,VIRGINIA,VA,BLAND,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,3479,20220315,ELECTION DAY,1707
2,70509,2020,VIRGINIA,VA,BLAND,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,3479,20220315,ELECTION DAY,1
3,71947,2020,VIRGINIA,VA,WILLIAMSBURG CITY,51830,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,4035,6890,20220315,ABSENTEE,-2694
4,71947,2020,VIRGINIA,VA,WILLIAMSBURG CITY,51830,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,4035,6890,20220315,ABSENTEE,-106


# Read in and clean population data

In [10]:
demo_2020 = pd.read_csv("/content/project_voting/data/county_data/0002_ds249_20205_county_E.csv", encoding='latin1')

row = demo_2020.iloc[0]  # Row 0 contains the descriptions of the variables

# Prints out each cell to show variables but honestly easier to just look at the codebook
print("Variables in Dataset:")

Variables in Dataset:


<ipython-input-10-4d1738020cf8>:1: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,

In [11]:
demo_2020 = demo_2020[demo_2020["STUSAB"] == "VA"] # Sorts the df for just VA counties. Can change if we want to look at US as a whole
demo_2020.head()

demo_2020= demo_2020.loc[:,("COUNTY","AMPWE001","AMPWE002","AMPWE003","AMPWE004","AMPWE005",
                     "AMPWE006","AMPWE007","AMPWE008","AMTCE001","AMPLE001","AMPLE002",
                     "AMPLE003")]  #Selects variables we want in the clean df

new_column_names = {"AMPWE001" : "Total Population",
        "AMPWE002":     "White alone",
        "AMPWE003":    "Black or African American alone",
        "AMPWE004":     "American Indian and Alaska Native alone",
        "AMPWE005":     "Asian alone",
        "AMPWE006":     "Native Hawaiian and Other Pacific Islander alone",
        "AMPWE007":     "Some other race alone",
        "AMPWE008":     "Two or more races",
        "AMTCE001":     "Per Capita income in the past 12 months",
        "AMPLE001":     "Median age: Total",
        "AMPLE002":     "Median age: Male",
        "AMPLE003":     "Median age: Female",
}  #These are all taken from the codebook in the github. MAKE SURE YOU'RE LOOKING AT THE RIGHT CODEBOOK FOR THE RIGHT CSV. I didnt realize that until way too late.

demo_2020.rename(columns=new_column_names, inplace=True)
demo_2020.head()

,COUNTY,Total Population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
2822,Accomack County,32560,21773,9390,48,244,0,154,951,27012,45.8,42.6,47.1
2823,Albemarle County,108819,87871,10005,315,5981,48,989,3610,46241,39.4,37.7,40.7
2824,Alleghany County,15030,13925,790,0,10,0,64,241,28048,48.0,46.1,50.0
2825,Amelia County,12970,9789,2876,0,1,0,0,304,32799,45.0,44.4,48.6
2826,Amherst County,31782,24330,5758,243,209,12,203,1027,28866,44.9,42.0,46.0


# Merge population and voting data

In [12]:
data_2020= data_2020.rename(columns={'county_name': 'COUNTY'})
data_2020['COUNTY'] = data_2020['COUNTY'].str.lower()
data_2020.head()

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.lower()
demo_2020.head()

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.replace(' ', '')
demo_2020['COUNTY'] = demo_2020['COUNTY'].str.replace('county', '')
data_2020['COUNTY'] = data_2020['COUNTY'].str.replace(' ', '')

demo_2020['COUNTY'] = demo_2020['COUNTY'].str.strip()
data_2020['COUNTY'] = data_2020['COUNTY'].str.strip()
merged_df_2020 = pd.merge(data_2020, demo_2020, on="COUNTY")
merged_df_2020.head()

,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
0,70509,2020,VIRGINIA,VA,bland,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,...,250,0,13,0,11,93,23664,46.1,41.9,51.3
1,70509,2020,VIRGINIA,VA,bland,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,...,250,0,13,0,11,93,23664,46.1,41.9,51.3
2,70509,2020,VIRGINIA,VA,bland,51021,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,1925,...,250,0,13,0,11,93,23664,46.1,41.9,51.3
3,71947,2020,VIRGINIA,VA,williamsburgcity,51830,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,4035,...,2285,36,997,0,116,827,32878,25.7,25.5,25.8
4,71947,2020,VIRGINIA,VA,williamsburgcity,51830,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,4035,...,2285,36,997,0,116,827,32878,25.7,25.5,25.8


# Regressions

In [18]:
y = merged_df_2020['difference']
vars = ['Median age: Total','Per Capita income in the past 12 months','White alone','Black or African American alone','American Indian and Alaska Native alone','Asian alone','Native Hawaiian and Other Pacific Islander alone','Some other race alone','Two or more races'] # This is a list of variables to use

X = merged_df_2020.loc[:,vars]

In [19]:
from sklearn.linear_model import LinearRegression # Import linear regression model
reg = LinearRegression(fit_intercept=False).fit(X, y) # Fit the linear model
results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
print('R-squared: ', reg.score(X, y)) # R squared measure
results

R-squared:  0.33328142844215713


,variable,coefficient
0,Median age: Total,122.942335
1,Per Capita income in the past 12 months,-0.162052
2,White alone,0.071016
3,Black or African American alone,-0.206144
4,American Indian and Alaska Native alone,-3.518820
5,Asian alone,-0.504218
6,Native Hawaiian and Other Pacific Islander alone,3.173719
7,Some other race alone,-0.396164
8,Two or more races,0.620042
